In [1]:
import json
from datetime import datetime, timedelta
import matplotlib.pylab as plot
import numpy as np

In [2]:
# Read data from http Zeek (formerly known as Bro) logs
with open("http.log",'r') as infile:
    file_data = infile.read()
    
# Split file by newlines
file_data = file_data.split('\n')

# Remove comment lines
http_data = []
for line in file_data:
    if line[0] is not None and line[0] != "#":
        http_data.append(line)

In [3]:
# Let's stack uris
uris = {}
for line in http_data:
    if len(line.split('\t')) > 9:
        uri = line.split('\t')[9].split('?')[0].split('&')[0]
        if uri not in uris.keys():
            uris[uri] = 1
        else:
            uris[uri] += 1

print(json.dumps(uris,indent=2))

{
  "/ftv2lastnode.gif": 2, 
  "/ftv2mnode.gif": 2, 
  "/pics/play_button_27x27px.gif": 4, 
  "/led.asp": 2, 
  "/pics/gray_corner_rt_5x50px.gif": 4, 
  "/img/device.gif": 4, 
  "/ RTSP/1.": 5, 
  "/pics/gray_corner_lt_5x50px.gif": 4, 
  "/webserverconfig.asp": 1, 
  "/auth/logo2_516.gif": 5, 
  "/index.htm": 1, 
  "/syslogserverconfig.asp": 2, 
  "/logo2_516.gif": 1, 
  "/neighbor_cache_table.asp": 2, 
  "/generalinst.htm": 1, 
  "/view/temp.shtml": 2, 
  "/img/checkbox_nchk.gif": 1, 
  "/jscript/sysstatus.js": 1, 
  "/SetModSerial.html": 1, 
  "/logo3.gif": 2, 
  "/status.jsp": 1, 
  "/port_setting.asp": 1, 
  "/syslog_message.asp": 1, 
  "/logo2_EDS-508A.gif": 1, 
  "/port_setting_show.asp": 1, 
  "/jscript/statistics.js": 3, 
  "/images/off.gif": 3, 
  "/pics/line_corner_rb_5x5px.gif": 4, 
  "/sysstatus.asp": 1, 
  "/overview.asp": 4, 
  "/jscript/powerconfig.js": 1, 
  "/jscript/login.js": 4, 
  "/mac_address_table_setting.asp": 4, 
  "/.git/HEAD": 11, 
  "/setid.html": 1, 
  "/ne

In [4]:
# Let's stack user agents
user_agents = {}
for line in http_data:
    if len(line.split('\t')) > 12:
        user_agent = line.split('\t')[11]
        if user_agent not in user_agents.keys():
            user_agents[user_agent] = 1
        else:
            user_agents[user_agent] += 1

print(json.dumps(user_agents,indent=2))

{
  "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0": 327, 
  "Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)": 171, 
  "-": 103, 
  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.64 Safari/537.36": 5045, 
  "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0": 12, 
  "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:41.0) Gecko/20100101 Firefox/41.0": 99, 
  "Wget/1.16.1 (linux-gnu)": 1
}


In [5]:
# Let's search for the nmap user agent
suspicious_user_agents = ['Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)']
nmap_scanned_hosts = {}
for line in http_data:
    if len(line.split('\t')) > 12:
        timestamp = line.split('\t')[0]
        client = line.split('\t')[2]
        server = line.split('\t')[4]
        user_agent = line.split('\t')[11]
        if user_agent in suspicious_user_agents:
            if client not in nmap_scanned_hosts.keys():
                nmap_scanned_hosts[client] = {server:{timestamp:1}}
            elif server not in nmap_scanned_hosts[client].keys():
                nmap_scanned_hosts[client][server] = {timestamp: 1}
            elif timestamp not in nmap_scanned_hosts[client][server].keys():
                nmap_scanned_hosts[client][server][timestamp] = 1
            else:
                nmap_scanned_hosts[client][server][timestamp] += 1

print(json.dumps(nmap_scanned_hosts,indent=2))

{
  "192.168.2.42": {
    "192.168.88.115": {
      "1445425464.684730": 1, 
      "1445425489.066291": 1, 
      "1445425456.492019": 1, 
      "1445425472.897110": 1, 
      "1445425505.330748": 1, 
      "1445425497.221008": 1, 
      "1445425472.798104": 1, 
      "1445425464.734434": 1, 
      "1445425489.264708": 1, 
      "1445425481.058994": 1, 
      "1445425456.491738": 1, 
      "1445425456.492152": 1, 
      "1445425464.684854": 1, 
      "1445425521.550031": 1, 
      "1445425456.491596": 1, 
      "1445425456.492557": 1, 
      "1445425513.438493": 1, 
      "1445425480.908743": 1
    }
  }, 
  "192.168.2.64": {
    "192.168.88.25": {
      "1445422296.875484": 1, 
      "1445422290.967679": 1, 
      "1445422289.381463": 1, 
      "1445422289.591706": 1, 
      "1445422290.459930": 1, 
      "1445422323.002866": 1, 
      "1445422289.808332": 1, 
      "1445422291.185004": 1, 
      "1445422290.239258": 1, 
      "1445422296.668006": 1, 
      "1445422290.239120": 1, 
  

In [6]:
# Add up the number of requests the client made to the server
print("client ip,server ip,num requests")
suspicious_hosts = {}
for client in sorted(nmap_scanned_hosts.keys()):
    for server in sorted(nmap_scanned_hosts[client].keys()):
        print(client + "," + server + "," + str(len(nmap_scanned_hosts[client][server])))
        if client not in suspicious_hosts.keys():
            suspicious_hosts[client] = [server]
        else:
            suspicious_hosts[client].append(server)

client ip,server ip,num requests
192.168.2.42,192.168.88.115,18
192.168.2.64,192.168.88.100,5
192.168.2.64,192.168.88.115,15
192.168.2.64,192.168.88.20,14
192.168.2.64,192.168.88.25,18
192.168.2.64,192.168.88.49,14
192.168.2.64,192.168.88.51,39
192.168.2.64,192.168.88.60,17
192.168.2.64,192.168.88.61,15
192.168.2.64,192.168.88.95,16


In [7]:
# Write CSV file out for display/distribution in excel
with open('suspicious_http_records.csv','w') as outfile:
    outfile.write("ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,host,uri,referrer,user_agent,request_body_len,response_body_len,status_code,status_msg,info_code,info_msg,filename,tags,username,password,proxied,orig_fuids,orig_mime_types,resp_fuids,resp_mime_types\n")
    for line in http_data:
        if len(line.split('\t')) > 12:
            timestamp = line.split('\t')[0]
            client = line.split('\t')[2]
            server = line.split('\t')[4]
            user_agent = line.split('\t')[11]
            uri = line.split('\t')[9]
            if client in suspicious_hosts.keys():
                if server in suspicious_hosts[client]:
                    outfile.write("\"" + line.replace("\t","\",\"") + "\"\n")
